In [1]:
!pip uninstall qiskit qiskit-aer qiskit-algorithms qiskit-optimization -y

# Step 2: Install compatible versions (tested combination)
!pip install qiskit==1.0.2
!pip install qiskit-aer==0.14.2
!pip install qiskit-algorithms==0.3.0
!pip install qiskit-optimization==0.6.1

Found existing installation: qiskit 1.0.2
Uninstalling qiskit-1.0.2:
  Successfully uninstalled qiskit-1.0.2
Found existing installation: qiskit-aer 0.14.2
Uninstalling qiskit-aer-0.14.2:
  Successfully uninstalled qiskit-aer-0.14.2
  Using cached qiskit-1.0.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached qiskit-1.0.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit-ibm-runtime 0.41.1 requires qiskit>=1.4.1, but you have qiskit 1.0.2 which is incompatible.
  Using cached qiskit_aer-0.14.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Using cached qiskit_aer-0.14.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Using cached qiskit_algorithms-0.3.0-py3-none-any.whl.metadata (4.2 kB)
Using cached qi

In [5]:
import numpy as np
from qiskit_aer import AerSimulator
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.applications import Tsp
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

# 1) Problem setup (cities & costs)
cities = {
    "SFO": (0.0, 0.0),
    "SEA": (1.0, 3.0),
    "DEN": (4.0, 2.5),
    "DFW": (6.0, 0.5),
}

ticket_price_matrix = np.array([
    [0,   200, 180, 220],   # From SFO
    [200, 0,   160, 190],   # From SEA
    [180, 160, 0,   210],   # From DEN
    [220, 190, 210, 0]      # From DFW
])

passenger_matrix = np.array([
    [0,   90, 120, 150],   # From SFO
    [100, 0,   80,  130],  # From SEA
    [110, 95,  0,   100],  # From DEN
    [140, 120, 105, 0]     # From DFW
])

labels = list(cities.keys())
n = len(labels)

# Euclidean distance (make it look realistic, scaled to ~km)
coords = np.array([cities[c] for c in labels])
dist = np.linalg.norm(coords[:, None, :] - coords[None, :, :], axis=2) * 500

# Economics Example
fuel_price = 0.9
fuel_burn_per_km = 2.5

# Weights: more emphasis on the fuel then the revenue
w_fuel = 0.8
w_rev  = 0.2

# Build edge cost matrix: w1*fuel_cost - w2*ticket_revenue
fuel_cost = fuel_price * fuel_burn_per_km * dist
ticket_rev_matrix = ticket_price_matrix * passenger_matrix
#negative because we are minimizing
edge_cost = -(w_rev * ticket_rev_matrix - w_fuel * fuel_cost)
np.fill_diagonal(edge_cost, 0.0)

# 2) Build TSP -> Quadratic Program (QP)
tsp = Tsp(edge_cost)
qp = tsp.to_quadratic_program()

# Convert to QUBO (binary form)
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)

# 3) Solve with QAOA on AerSimulator
sampler = Sampler()
qaoa = QAOA(sampler=sampler, reps=2, optimizer=COBYLA(maxiter=150))
meo = MinimumEigenOptimizer(qaoa)
result = meo.solve(qubo)

# 4) Result
tsp_solution = tsp.interpret(result)
route_indices = tsp_solution
route_labels = [labels[i] for i in route_indices] + [labels[route_indices[0]]]

total_weighted_cost = 0.0
total_fuel = 0.0
total_revenue = 0.0

for a, b in zip(route_indices, route_indices[1:] + [route_indices[0]]):
    total_weighted_cost += edge_cost[a, b]
    total_fuel += fuel_cost[a, b]
    total_revenue += ticket_rev_matrix[a, b]

profit = total_revenue - total_fuel

print("\n=== Weighted Optimization Results ===")
print("Best route (labels):", " -> ".join(route_labels))
print("Total weighted cost (optimization objective):", round(float(total_weighted_cost), 2))
print("Total fuel cost ($):", round(float(total_fuel), 2))
print("Total ticket revenue ($):", round(float(total_revenue), 2))
print("Net Profit ($):", round(float(profit), 2))

print("\nLeg breakdown (fuel_cost, ticket_revenue, weighted_cost):")
for a, b in zip(route_indices, route_indices[1:] + [route_indices[0]]):
    fc = float(fuel_cost[a, b])
    tr = float(ticket_rev_matrix[a, b])
    wc = float(edge_cost[a, b])
    print(f"{labels[a]} -> {labels[b]} : fuel ${fc:,.0f}, tickets ${tr:,.0f}, weighted {wc:,.0f}")


=== Weighted Optimization Results ===
Best route (labels): DFW -> DEN -> SEA -> SFO -> DFW
Total weighted cost (optimization objective): -4502.4
Total fuel cost ($): 16934.49
Total ticket revenue ($): 90250.0
Net Profit ($): 73315.51

Leg breakdown (fuel_cost, ticket_revenue, weighted_cost):
DFW -> DEN : fuel $3,182, tickets $22,050, weighted -1,864
DEN -> SEA : fuel $3,422, tickets $15,200, weighted -303
SEA -> SFO : fuel $3,558, tickets $20,000, weighted -1,154
SFO -> DFW : fuel $6,773, tickets $33,000, weighted -1,181
